# Data processing and storage
The following sections will handle the scraped data stored in json files. We start by opening our data file, in our case it's raw_study_in_germany_data.json

In [1]:
# Extracting the crawler's data from the output file
import rootutils
notebook_path = "../notebooks/scraping_cleaning_pipeline.ipynb"
rootutils.setup_root(search_from=notebook_path, indicator='.project_root', pythonpath=True)

import json
file_path = '../temporary_folder/output'

with open(file_path+'.json', 'r', encoding='utf-8') as file:
    raw_data = json.load(file)

We create functions to handle and remove personal information from the raw scraped data. We also make an LLM call (OpanAI API) for further processing and to define whether the resulting information is relevant to students who would like to study/live in Germany.

Before running this code, make sure to run `python -m spacy download en_core_web_sm` in your terminal.

In [5]:
!pip install spacy
!python -m spacy download en_core_web_sm

  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached marisa_trie-1.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (9.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.4/634.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.9/844.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.8 MB/s eta 0:00:00a 0:00:01
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached marisa_trie-1.2.1-cp310-cp310-macosx_11_0_arm64.whl (174 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --up

In [12]:
import json
import re
import os
import pandas as pd
from openai import OpenAI
import spacy

# Optional: Import OpenAI if using LLM-based cleaning
# try:
#     OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # Ensure your API key is set as an environment variable
#     client = OpenAI()
# except ImportError:
#     OpenAI = None
#     print("OpenAI library not found. LLM-based cleaning will be skipped.")

# Initialize spaCy model
nlp = spacy.load("en_core_web_sm")

def remove_emails(text):
    """Remove email addresses from text."""
    email_pattern = r'\S+@\S+'
    return re.sub(email_pattern, '[EMAIL]', text)

def remove_phone_numbers(text):
    """Remove phone numbers from text."""
    phone_pattern = r'\+?\d[\d -]{8,}\d'
    return re.sub(phone_pattern, '[PHONE]', text)

def remove_names(text):
    """Remove person names using spaCy's NER."""
    doc = nlp(text)
    entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents if ent.label_ == 'PERSON']
    cleaned_text = text
    # Replace names with [NAME]
    for start, end, label in reversed(entities):
        cleaned_text = cleaned_text[:start] + '[NAME]' + cleaned_text[end:]
    return cleaned_text

def remove_addresses(text):
    """Remove addresses using regex and spaCy."""
    # Simple regex pattern for addresses (can be enhanced)
    address_pattern = r'\d{1,5}\s\w+\s(?:Street|St|Avenue|Ave|Boulevard|Blvd|Road|Rd|Lane|Ln|Drive|Dr)\b[\w\s,.-]*'
    text = re.sub(address_pattern, '[ADDRESS]', text)
    
    # Using spaCy to remove GPE (Geopolitical Entities) as a proxy for locations
    doc = nlp(text)
    entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    cleaned_text = text
    # Replace locations with [LOCATION]
    for start, end, label in reversed(entities):
        cleaned_text = cleaned_text[:start] + '[LOCATION]' + cleaned_text[end:]
    return cleaned_text

def remove_personal_info(text):
    """Aggregate function to remove various personal information."""
    text = remove_emails(text)
    text = remove_phone_numbers(text)
    text = remove_names(text)
    text = remove_addresses(text)
    return text

# def define_relevance_with_llm(text):
    relevance_prompt=f"""
    You are an automated classifier designed to evaluate whether a given text 
    contains relevant information for students interested in studying and living
    in Germany. Your task is to read the provided text and determine its relevancy
    based on the following criteria:

    Respond with "Yes" if the text includes information such as:

    Study opportunities (e.g., universities, courses, scholarships)
    Living conditions (e.g., cost of living, accommodation tips)
    Cultural insights (e.g., cultural norms, language tips)
    Practical advice for adapting to life in Germany
    Local resources and support systems for international students
    Respond with "No" if the text:

    Does not pertain to studying or living in Germany
    Contains irrelevant or unrelated information
    Focuses on personal anecdotes without broader applicability
    Discusses topics outside the scope of student life in Germany
    Examples:

    Text: "Discover the top universities in Germany offering scholarships
    for international students. Learn about the application process and tips
    for living on a student budget."

    Response: Yes

    Text: "The weather today is sunny with a high of 25°C. It's a perfect day
    for a picnic in the park."

    Response: No

    Text: "Navigating the German public transportation system can be challenging
    for newcomers. Here's a guide to help you get around efficiently."

    Response: Yes

    Text: "I went hiking in the Alps last summer and it was an unforgettable experience."

    Response: No

    Now, evaluate the following text:

    Text: "{text}"

    Response:
    """
    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": relevance_prompt},
        ])
        
        relevance = response.choices[0].message.content
        return relevance
    except Exception as e:
        print(f"Error during LLM-based relevance detection: {e}")
        return text
    

# def clean_text_with_llm(text):
    """
    Clean text using an LLM like OpenAI's GPT.
    This function sends a prompt to the LLM to clean the text.
    Ensure you have set the OPENAI_API_KEY environment variable.
    """
    if not client:
        print("OpenAI library not available. Skipping LLM-based cleaning.")
        return text
    
    cleaning_prompt = f"""
    You are an expert data cleaner tasked with processing scraped website data 
    to create clear and informative content for students interested in studying
    and living in Germany. The input text may contain disorganized information and 
    personal details. Your objectives are to:

    Extract Relevant Information:

    Identify and retain only the information pertinent to studying and living in 
    Germany, such as educational opportunities, living conditions, cultural insights,
    accommodation tips, and local resources.
    
    Remove Personal Information:

    Exclude all personal data, including names, contact details, addresses, email addresses,
    phone numbers, and any other identifying information.
    Ensure Coherence and Clarity:

    Organize the extracted information into a well-structured, coherent, and readable format.
    Maintain the original meaning and intent of the content without adding or omitting critical information.
    
    Original Text: "{text}"

    Cleaned Text:
    """
    
    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": cleaning_prompt},
        ])
        
        cleaned_text = response.choices[0].message.content
        return cleaned_text
    except Exception as e:
        print(f"Error during LLM-based cleaning: {e}")
        return text

We use all our previous functions to clean the data. Then store it in a separate json file.

In [15]:

def main(INPUT_FILE = f'{file_path}.json',
    CLEANED_FILE = f'{file_path}_cleaned.json'):
    if not os.path.exists(INPUT_FILE):
        print(f"Input file '{INPUT_FILE}' not found.")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            print(f"Loaded {len(data)} data points from '{INPUT_FILE}'.")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return

    df = pd.DataFrame(data)

    print("\nInitial DataFrame Info:")
    print(df.info())

    required_columns = ['url', 'title', 'text_content']
    for col in required_columns:
        if col not in df.columns:
            print(f"Missing required column: '{col}'. Please check your data.")
            return

    initial_count = len(df)
    df.drop_duplicates(subset=['url'], inplace=True)
    duplicates_removed = initial_count - len(df)
    print(f"\nRemoved {duplicates_removed} duplicate entries based on 'url'.")

    initial_count = len(df)
    df.dropna(subset=['url', 'text_content'], inplace=True)
    missing_removed = initial_count - len(df)
    print(f"Removed {missing_removed} entries with missing 'url' or 'text_content'.")

    titles_filled = df['title'].isnull().sum()
    df['title'].fillna('No Title', inplace=True)
    print(f"Filled {titles_filled} missing 'title' entries with 'No Title'.")

    print("\nRemoving personal information...")
    df['text_content'] = df['text_content'].apply(remove_personal_info)

    apply_llm = False 
    if apply_llm:
        print("Applying LLM-based cleaning...")
        df['text_content'] = df['text_content'].apply(clean_text_with_llm)

    df.to_json(CLEANED_FILE, orient='records', indent=4)
    print(f"\nCleaned data saved to '{CLEANED_FILE}'.")

In [16]:
main()

Loaded 712 data points from '../temporary_folder/output.json'.

Initial DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           712 non-null    object
 1   title         712 non-null    object
 2   text_content  712 non-null    object
dtypes: object(3)
memory usage: 16.8+ KB
None

Removed 22 duplicate entries based on 'url'.
Removed 0 entries with missing 'url' or 'text_content'.
Filled 0 missing 'title' entries with 'No Title'.

Removing personal information...


/var/folders/ns/0spd8fg95tv2s7hys988_pb00000gp/T/ipykernel_30125/509171924.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['title'].fillna('No Title', inplace=True)


KeyboardInterrupt: 

At this point we have a cleaned json file, we can now convert it to a txt file for storage in a vector database (Alternatively, we can pause here and return later since our data is present locally)

In [2]:
from datetime import datetime

with open(f'{file_path}_cleaned.json', 'r', encoding='utf-8') as file:
    clean_data = json.load(file)


In [3]:
output_json_path = f"{file_path}_for_qdrant.json"

if isinstance(clean_data, list):
    processed_data = []
    for entry in clean_data:
        if isinstance(entry, dict) and 'text_content' in entry:
            processed_entry = {
                'text': entry['text_content'],
                'metadata': {
                    'url': entry['url'],
                    'title': entry['title'],
                    'source_type': 'website',
                    'date_added': datetime.now().isoformat()
                }
            }
            processed_data.append(processed_entry)
            
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(processed_data, json_file, indent=4)

We can now store the data in a vector database. We are using QDRANT.

In [9]:
from app.utils.storage_utils import query_qdrant
from scripts.data_preparation import store_in_qdrant
import qdrant_client
import os

# Get environment variables for Qdrant configuration
QDRANT_URL = os.environ['QDRANT_URL']
ENVIRONMENT = os.environ['ENVIRONMENT']

# Initialize Qdrant client based on environment
if ENVIRONMENT == 'dev':
    # For local development, connect to localhost
    client = qdrant_client.QdrantClient(url="http://localhost:6333")
else:
    QDRANT_API_KEY = os.environ['QDRANT_API_KEY']
    # For production, connect to cloud instance using URL and API key
    client = qdrant_client.QdrantClient(
        QDRANT_URL,
        api_key=QDRANT_API_KEY
    )

In [11]:
store_in_qdrant(client, "study-in-germany", file_path="../temporary_folder/handbook_germany_qdrant.json")

Generating embeddings: 100%|██████████| 175/175 [00:07<00:00, 23.88it/s]


We now query the vector database to test if it's working. The LLM will return the most relevant information based on the query. Any information that is not relevant to the query will be ignored.

In [9]:
query_qdrant(client, "study-in-germany", "On average how much do student working jobs pay in computer science ?")

{'context': '**Studying and Living in Germany: Key Information for Students**\n\n**Student Jobs and Working Regulations**\n\n1. **Working Hours:** \n   - If you are on a student visa, you can work up to 120 full days or 240 half days per year. From March 1, 2024, this will increase to 140 full days or 280 half days annually.\n   - During the lecture period, it is advisable to not exceed 20 working hours per week to maintain your student status.\n\n2. **Types of Student Jobs:**\n   - Common student jobs can be found in sectors such as catering (waitstaff), retail (sales assistants), and education (tutors).\n   - You can also work as a “student trainee” (Werkstudent) or as a student assistant ("Hiwi") within your university.\n   - Additional resources for job searches include websites like jobmensa.de, studentjob.de, and nebenjob.de.\n\n3. **Earnings and Social Security:**\n   - Students earning less than €538 per month are categorized as having a "Mini-job" and generally do not need to 

In [13]:
result = query_qdrant(client, "study-in-germany", "What rental platform should i use to find an apartment?")

In [17]:
result['sources'][0]

{'url': 'https://handbookgermany.de/en/search-a-flat',
 'title': 'Searching for a Flat ',
 'source_type': 'website',
 'date_added': '2024-12-19T11:55:11.082255'}

In [10]:
query_qdrant(client, "study-in-germany", "What is the capital of Algeria")

{'answer': 'Empty Response', 'sources': [], 'scores': []}

In [7]:
from app.utils.storage_utils import delete_from_qdrant_by_url

delete_from_qdrant_by_url(client, "study-in-germany", "handbookgermany.de")

{'success': True,
 'message': "Deleted 100 entries with URLs containing 'handbookgermany.de'"}